# Segmenting and Clustering Neighborhoods in Toronto

# Part 1: Create Toronto neighbourhood dataframe

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', {'class': 'wikitable sortable'}).tbody

In [4]:
rows = table.find_all('tr')
columns = [v.text.replace('\n', '') for v in rows[0].find_all('th')]
print(columns)

['Postcode', 'Borough', 'Neighbourhood']


In [5]:
df = pd.DataFrame(columns = columns)
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    if len(tds) == 2:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', '').replace('\xa0','')]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
    
    df = df.append(pd.Series(values, index=columns), ignore_index=True)

df.head()
df.shape

(288, 3)

In [6]:
df = df[df.Borough!='Not assigned']
df.head()
df.shape

(211, 3)

In [7]:
df_Toronto = df.groupby(by=['Postcode', 'Borough']).agg(lambda x:','.join(x))
df_Toronto.reset_index(inplace=True)
df_Toronto.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
df_Toronto.shape

(103, 3)

In [9]:
import numpy as np
df_Toronto['Neighbourhood'] = np.where(df_Toronto.Neighbourhood == "Not assigned", df_Toronto.Borough, df_Toronto.Neighbourhood)
#another option, not using numpy
#df_Toronto.loc[df_Toronto['Neighbourhood'] == "Not assigned"] = df_Toronto['Borough']

df_Toronto.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
df_Toronto.shape

(103, 3)

# Part 2: Create Toronto neighbourhood dataframe with geographical coordinates

In [11]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
df_geo.shape

(103, 3)

In [13]:
df_Toronto.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
df_Toronto.shape

(103, 3)

In [16]:
df_torgeo = pd.merge(df_Toronto, df_geo, how='inner', left_on='Postcode', right_on='Postal Code')
df_torgeo.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [17]:
df_torgeo.shape

(103, 6)

In [18]:
df_torgeo.drop(['Postal Code'], axis = 1, inplace =True)
df_torgeo.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
df_torgeo.rename(columns={'Postcode': 'PostalCode'},inplace=True)
print(df_torgeo.shape)
df_torgeo.head()

(103, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 3: Clustering of Toronto neighbourhood

In [34]:
!conda install -c conda-forge geopy --yes        
!conda install -c conda-forge folium=0.5.0 --yes 

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


from geopy.geocoders import Nominatim 
GeoLocator = Nominatim(user_agent='My-IBMNotebook')# convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Admin\Anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.3.9           |           py37_0         149 KB  conda-forge
    conda-4.7.5                |           py37_0         3.0 MB  conda-forge
    conda-package-handling-1.3.10|           py37_0         277 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

  conda-package-han~ conda-forge/win-64::conda-package-handling-1.3.10-py37_0
  geographiclib      conda-forge/noarch::geographiclib-1.4



==> WARNING: A newer version of conda exists. <==
  current version: 4.6.11
  latest version: 4.7.5

Please update conda by running

    $ conda update -n base -c defaults conda


'ET' is not recognized as an internal or external command,
operable program or batch file.
WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.


Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Admin\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.1.0               |           py37_0         721 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         819 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/win-64::altair-3.1.0-py37_0
  branca             conda-forge/noarch::branca-0.3.1-py_0
  folium             conda-forge/noarch::folium-0.5.0-py_0
  vincent            conda-for

WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.


Libraries imported.


In [35]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [36]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_torgeo['Latitude'], df_torgeo['Longitude'], df_torgeo['Borough'], df_torgeo['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [37]:
df_Toronto['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [38]:
Nyork_data = df_torgeo[df_torgeo['Borough'] == 'North York'].reset_index(drop=True)
Nyork_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills,York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook,Willowdale",43.789053,-79.408493


In [39]:
address = 'North York, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.7708175, -79.4132998.


In [40]:
map_nyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Nyork_data['Latitude'], Nyork_data['Longitude'], Nyork_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyork)  
    
map_nyork

In [41]:
CLIENT_ID = 'I3MSQA3RMLAT4DXOEXPBYN5DPA1TIYJZPKTGMS2413WYS33R' # your Foursquare ID
CLIENT_SECRET = 'KO3BSWWDBX20QXWNIZA10KCWTWPGN1KOSNYEAE5ZX1S2ITXL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: I3MSQA3RMLAT4DXOEXPBYN5DPA1TIYJZPKTGMS2413WYS33R
CLIENT_SECRET:KO3BSWWDBX20QXWNIZA10KCWTWPGN1KOSNYEAE5ZX1S2ITXL


In [42]:
Nyork_data.loc[0, 'Neighbourhood']

'Hillcrest Village'

In [43]:
neighbourhood_latitude = Nyork_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = Nyork_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = Nyork_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Hillcrest Village are 43.8037622, -79.3634517.


In [44]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 700 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=I3MSQA3RMLAT4DXOEXPBYN5DPA1TIYJZPKTGMS2413WYS33R&client_secret=KO3BSWWDBX20QXWNIZA10KCWTWPGN1KOSNYEAE5ZX1S2ITXL&v=20180605&ll=43.8037622,-79.3634517&radius=700&limit=100'

In [45]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d171437ad1789002c38d3aa'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.81006220630001,
    'lng': -79.35473877952963},
   'sw': {'lat': 43.79746219369999, 'lng': -79.37216462047037}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd9842be914a593adbd56fa',
       'name': 'Tastee',
       'location': {'address': '3913 Don Mills Rd.',
        'crossStreet': 'at Cliffwood Rd.',
        'lat': 43.80772211146167,
        'lng': -79.35679781099806,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80772211146167,
          'lng': -79.35679781099806}],
        'distance': 692,
        'post

In [46]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [47]:

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Tastee,Bakery,43.807722,-79.356798
1,Purple Sageway,Housing Development,43.809332,-79.364604
2,New Greattime Corp.,Chinese Restaurant,43.807414,-79.356717
3,Pickle Barrel,Diner,43.799588,-79.369093


In [48]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [49]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [50]:
nyork_venues = getNearbyVenues(names=Nyork_data['Neighbourhood'],
                                   latitudes=Nyork_data['Latitude'],
                                   longitudes=Nyork_data['Longitude']
                                  )

Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park,Lawrence Manor East
Lawrence Heights,Lawrence Manor
Glencairn
Downsview,North Park,Upwood Park
Humber Summit
Emery,Humberlea


In [51]:
print(nyork_venues.shape)
nyork_venues.head()

(251, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store


In [52]:
nyork_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",25,25,25,25,25,25
"CFB Toronto,Downsview East",3,3,3,3,3,3
Don Mills North,5,5,5,5,5,5
Downsview Central,4,4,4,4,4,4
Downsview Northwest,5,5,5,5,5,5
Downsview West,3,3,3,3,3,3
"Downsview,North Park,Upwood Park",5,5,5,5,5,5


In [53]:
print('There are {} uniques categories.'.format(len(nyork_venues['Venue Category'].unique())))

There are 111 uniques categories.


In [54]:
# one hot encoding
nyork_onehot = pd.get_dummies(nyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
nyork_onehot['Neighbourhood'] = nyork_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [nyork_onehot.columns[-1]] + list(nyork_onehot.columns[:-1])
nyork_onehot = nyork_onehot[fixed_columns]

nyork_onehot.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fraternity House,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Other Repair Shop,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Fairview,Henry Farm,Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [55]:
nyork_onehot.shape

(251, 112)

In [56]:
nyork_grouped = nyork_onehot.groupby('Neighbourhood').mean().reset_index()
nyork_grouped

,Neighbourhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fraternity House,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Other Repair Shop,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor,Downsview North,Wilson Heights",0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.055556,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.00,0.0,0.0,0.000000,0.00,0.055556,0.000000,0.000000,0.000000,0.055556,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.055556,0.000000,0.000000,0.00,0.0000,0.055556,0.055556,0.0000,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.00,0.00,0.00,0.000000,0.055556,0.000000,0.055556,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.000000,0.00,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000
1,Bayview Village,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.0000,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park,Lawrence Manor East",0.0000,0.000000,0.040000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.0,0.040000,0.000000,0.000000,0.000000,0.000000,0.080000,0.04,0.0,0.0,0.000000,0.04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.080000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.0000,0.000000,0.00,0.04,0.040000,0.000000,0.0,0.00,0.00,0.000000,0.040000,0.04,0.000000,0.00,0.080000,0

In [57]:
nyork_grouped.shape

(23, 112)

In [58]:

num_top_venues = 5

for hood in nyork_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = nyork_grouped[nyork_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor,Downsview North,Wilson Heights----
                 venue  freq
0          Coffee Shop  0.11
1                Diner  0.06
2        Shopping Mall  0.06
3           Restaurant  0.06
4  Fried Chicken Joint  0.06


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park,Lawrence Manor East----
                  venue  freq
0  Fast Food Restaurant  0.08
1    Italian Restaurant  0.08
2           Coffee Shop  0.08
3              Pharmacy  0.04
4        Sandwich Place  0.04


----CFB Toronto,Downsview East----
               venue  freq
0            Airport  0.33
1               Park  0.33
2  Other Repair Shop  0.33
3                Gym  0.00
4          Pet Store  0.00


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center   0.2
1  Caribbean Restaurant   0.2
2                  Café   0.2
3   

In [59]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [60]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = nyork_grouped['Neighbourhood']

for ind in np.arange(nyork_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nyork_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Supermarket,Middle Eastern Restaurant,Frozen Yogurt Shop,Pharmacy,Pizza Place,Deli / Bodega,Restaurant,Bridal Shop,Sandwich Place
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
2,"Bedford Park,Lawrence Manor East",Coffee Shop,Fast Food Restaurant,Italian Restaurant,Grocery Store,Indian Restaurant,Liquor Store,Cupcake Shop,Café,Pharmacy,Pizza Place
3,"CFB Toronto,Downsview East",Park,Airport,Other Repair Shop,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
4,Don Mills North,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega


In [61]:
# set number of clusters
kclusters = 5

nyork_grouped_clustering = nyork_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 3, 0, 3, 3, 3, 3, 3, 4])

In [62]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Clusters', kmeans.labels_)

nyork_merged = Nyork_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
nyork_merged = nyork_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

nyork_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,3.0,Dog Run,Mediterranean Restaurant,Pool,Golf Course,Fried Chicken Joint,Dim Sum Restaurant,Comfort Food Restaurant,Construction & Landscaping,Furniture / Home Store,Convenience Store
1,M2J,North York,"Fairview,Henry Farm,Oriole",43.778517,-79.346556,3.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Kids Store,Toy / Game Store,Tea Room,Metro Station,Bakery,Japanese Restaurant
2,M2K,North York,Bayview Village,43.786947,-79.385975,3.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
3,M2L,North York,"Silver Hills,York Mills",43.757490,-79.374714,2.0,Cafeteria,Women's Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
4,M2M,North York,"Newtonbrook,Willowdale",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nyork_merged['Latitude'], nyork_merged['Longitude'], nyork_merged['Neighbourhood'], nyork_merged['Clusters']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [64]:
nyork_merged.loc[nyork_merged['Clusters'] == 0, nyork_merged.columns[[1] + list(range(5, nyork_merged.shape[1]))]]

,Borough,Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,North York,0.0,Park,Convenience Store,Bank,Bar,Electronics Store,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
8,North York,0.0,Park,Food & Drink Shop,Fast Food Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega
13,North York,0.0,Park,Airport,Other Repair Shop,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store


In [65]:

nyork_merged.loc[nyork_merged['Clusters'] == 1, nyork_merged.columns[[1] + list(range(5, nyork_merged.shape[1]))]]

,Borough,Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,North York,1.0,Hockey Arena,Portuguese Restaurant,Intersection,Coffee Shop,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop


In [66]:
nyork_merged.loc[nyork_merged['Clusters'] == 2, nyork_merged.columns[[1] + list(range(5, nyork_merged.shape[1]))]]

,Borough,Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,2.0,Cafeteria,Women's Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop


In [67]:
nyork_merged.loc[nyork_merged['Clusters'] == 3, nyork_merged.columns[[1] + list(range(5, nyork_merged.shape[1]))]]

,Borough,Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3.0,Dog Run,Mediterranean Restaurant,Pool,Golf Course,Fried Chicken Joint,Dim Sum Restaurant,Comfort Food Restaurant,Construction & Landscaping,Furniture / Home Store,Convenience Store
1,North York,3.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Kids Store,Toy / Game Store,Tea Room,Metro Station,Bakery,Japanese Restaurant
2,North York,3.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
5,North York,3.0,Ramen Restaurant,Coffee Shop,Sushi Restaurant,Restaurant,Sandwich Place,Japanese Restaurant,Café,Pet Store,Bubble Tea Shop,Plaza
7,North York,3.0,Grocery Store,Pizza Place,Discount Store,Coffee Shop,Pharmacy,Asian Restaurant,Electronics Store,Convenience Store,Cosmetics Shop,Cupcake Shop
9,North York,3.0,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega
10,North York,3.0,Gym,Asian Restaurant,Coffee Shop,Beer Store,Grocery Store,Smoke Shop,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Discount Store
11,North York,3.0,Coffee Shop,Supermarket,Middle Eastern Restaurant,Frozen Yogurt Shop,Pharmacy,Pizza Place,Deli / Bodega,Restaurant,Bridal Shop,Sandwich Place
12,North York,3.0,Coffee Shop,Miscellaneous Shop,Caribbean Restaurant,Metro Station,Massage Studio,Bar,Falafel Restaurant,Discount Store,Cosmetics Shop,Cupcake Shop
14,North York,3.0,Grocery Store,Bank,Shopping Mall,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store


In [68]:
nyork_merged.loc[nyork_merged['Clusters'] == 4, nyork_merged.columns[[1] + list(range(5, nyork_merged.shape[1]))]]

,Borough,Clusters,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North York,4.0,Baseball Field,Women's Store,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop
